In [0]:
%sql 
WITH base AS (
  SELECT
    employee_id,
    employee_name,
    cost_center_id,
    legal_entity_id,
    salary,
    salary / 4 AS quarterly_salary,
    year(date_add(hire_date, seq)) AS employment_year,
    quarter(date_add(hire_date, seq)) AS employment_quarter
  FROM dim_employees
  LATERAL VIEW posexplode(
    sequence(
      0,
      datediff(
        COALESCE(termination_date, current_date()),
        hire_date
      )
    )
  ) AS seq, day_offset
  WHERE hire_date IS NOT NULL
  GROUP BY ALL
) 
SELECT 
  employment_year
  ,employment_quarter
  ,SUM(quarterly_salary)
FROM base
GROUP BY 1, 2
ORDER BY 1, 2

In [0]:
%sql
SELECT * 
FROM fact_fpa_actuals;


In [0]:
%sql
SELECT
  *
FROM stg_spend_transactions
LIMIT 100;

In [0]:
%sql
SELECT * 
FROM fact_fpa_forecasts; 

In [0]:
%sql
SELECT
  scen.fiscal_year
  ,scen.fiscal_quarter
  ,scen.scenario_key
  ,scen.cost_center_id
  ,scen.cost_center_name
  ,scen.legal_entity_id
  ,scen.legal_entity_name
  ,fore.forecast_amount
  ,act.actual_amount
  ,act.actual_amount / fore.forecast_amount AS forecast_accuracy
FROM dim_fpa_scenarios AS scen
LEFT JOIN fact_fpa_actuals AS act 
  ON scen.scenario_key = act.scenario_key
LEFT JOIN fact_fpa_forecasts AS fore
  ON scen.scenario_key = fore.scenario_key
ORDER BY 1, 2, scen.cost_center_id, scen.legal_entity_id

In [0]:
%sql
WITH actuals AS (
  SELECT
    fiscal_year
    ,fiscal_quarter
    ,SUM(actual_amount) AS actual_amount
  FROM fact_fpa_actuals
  GROUP BY ALL
), revenue AS (
  SELECT
    YEAR(transaction_date) AS fiscal_year
    ,QUARTER(transaction_date) AS fiscal_quarter
    ,SUM(amount) AS rev_amount
  FROM stg_revenue_transactions
  GROUP BY ALL
)
SELECT
  actuals.*
  ,revenue.rev_amount
FROM actuals
JOIN revenue
  ON actuals.fiscal_year = revenue.fiscal_year
  AND actuals.fiscal_quarter = revenue.fiscal_quarter
ORDER BY 1, 2